## Setup

In [2]:
# Setup
import geopandas as gpd
import flopy
import numpy as np
import os
import pandas as pd

In [3]:
cwd = '\\'.join(os.getcwd().split('\\')[:-2])

## Tests

### Test: Load the model

In [ ]:
# bug detected:
# 1. if the script is not in the same folder as the .nam file,
#    an error arises saying that the ._kx file is not found
# 2. if the .sfr file is in a SFR2 format (e.g. NSRTM = 2),
#    the file doesn't get read

m = flopy.mfusg.MfUsg.load(os.path.join(os.getcwd(), 't_m11_C1.nam'))

c:\Users\user\anaconda3\envs\maurice\Lib\site-packages\flopy\mbase.py:97: UserWarning: The program mfusg does not exist or is not executable.
  warn(


In [18]:
# m.sfr.reach_data

In [33]:
# m.sfr.segment_data

In [ ]:
sp = 1 #stress period number
m.sfr.segment_data[sp-1]

In [12]:
m.sfr.nstrm

np.int64(174)

In [14]:
m.sfr.isfropt

0

In [ ]:
m.sfr.isfropt = 2
#setting it from here doesn't change the output format

#### Test: load sfr condition from shapefile

In [4]:
sfr_shp = gpd.read_file(os.path.join(cwd, 'test_models', 'SFR_ICALC2.shp'))

In [6]:
sfr_shp.shape

(174, 254)

In [8]:
sfr_shp.Slope

0      0.000019
1      0.000019
2      0.000019
3      0.000019
4      0.000019
         ...   
169    0.000019
170    0.000019
171    0.000019
172    0.000019
173    0.000019
Name: Slope, Length: 174, dtype: float64

### Test: Modify the segment data parameters

In [89]:
# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti.csv'))
q[q.isna()] = 0

In [90]:
for sp in range(1, 91+1):
    tool = q.loc[q.SP == sp, ['Qin', 'Sg_2', 'Sg_3', 'Sg_4', 'Sg_5', 'Sg_6', 'Sg_7']].values[0]
    tool = np.around(tool, 5)
    tool = [tool[0]] + [-x if x != 0 else 0 for x in tool[1:]]
    m.sfr.segment_data[sp-1].flow = tool

### Test: write the new .sfr file

In [16]:
m.model_ws = os.path.join(cwd, 'test_models', 'tool_m11_c1', 'test_datasets_m11_c1')

In [19]:
m.write_input()